<h3 style="text-align:center"> Gather <b>Formula 1</b> data from <a href="http://ergast.com/mrd/">Ergast Developer API</a></h3>

Version 1.0.0  
Author: [Dorian ROUX](https://rouxdorian.com)  
Estimated Runtime: 13 minutes. 
 
---- 

#### Libraries

In [1]:
import itertools
import json
import os
import random
import requests

#### Functions

In [236]:
# - Display a sample of a dictionnary - #
def displayDictSample(dictionary: dict, numSample: int = 5):
    """Display a sample of a dictionnary.
    Args:
        dictionary (dict): the dictionnary to display.
        numSample (int, optional): the number of elements to display. Defaults to 5.
    """    
    key = random.sample(list(dictionary.keys()), k=numSample)
    str_ = f'Sample of the Dictionnary ({numSample} observations):'
    print(f"{str_}\n{'*' * len(str_)} \n")
    for k in sorted(key, reverse=False):
        print(k, dictionary[k])

# - Display a sample of a list - #
def displayListSample(listContent: list, numSample: int = 5):
    """Display a sample of a list.
    Args:
        listContent (list): the list to display.
        numSample (int, optional): the number of elements to display. Defaults to 5.
    """    
    str_ = f'Sample of the List ({numSample} observations):'
    print(f"{str_}\n{'*' * len(str_)} \n")
    for i in range(numSample):
        print(listContent[i])

# - Update a dictionnary - #
def updateDictionnary(inputDict: dict, outputDict: dict, inputKeys: list, outputKeys: list):
    """Update a dictionnary by adding a new key and value based on the existance of the inputKeys within the inputDict.
    Args:
        inputDict (dict): the input dictionnary.
        outputDict (dict): the output dictionnary.
        inputKeys (list): the list of keys to look for in the inputDict.
        outputKeys (list): the list of keys to add in the outputDict depending on the inputKeys.
    """    
    for i_, inputKey in enumerate(inputKeys):
        if inputKey in inputDict.keys():
            outputDict[outputKeys[i_]] = inputDict[inputKey]
    return outputDict

# - Get the inner dictionary of a dictionary - #
def getInnerDict(dictContent: dict, keys: list, defaultValue: str = ''):
    """Recursively get the inner dictionary of a dictionary based on a list of keys.
    Args:
        dictContent (dict): the dictionary to look into.
        keys (list): the list of keys to look for.
        defaultValue (str, optional): the default value to return if the key is not found. Defaults to ''.
    """    
    if len(keys) > 0:
        if keys[0] in dictContent.keys():
            dictContent = dictContent[keys[0]]
            return getInnerDict(dictContent, keys[1:], defaultValue=defaultValue)
        return defaultValue
    return dictContent

# - Make a request to the Ergast Developer API - #
def makeErgastRequest(requestContent: str, apiLimit: int = 1000, apiOffset: int = 0, apiExport: str = 'json'):
    """Make a request to the Ergast Developer API and return the response.
    Args:
        requestContent (str): the content of the request as a string. It is the content after the Ergast URL.
        apiLimit (int, optional): the number of information to request. Defaults to 1000.
        apiOffset (int, optional): the number of offset to setup. Defaults to 0.
        apiExport (str, optional): the type of exportation (JSON, XML). Defaults to 'json'.
    """    
    setupParameters = {'LIMIT' : apiLimit, 'OFFSET' : apiOffset, 'EXPORT' : apiExport}
    requestURL = f"http://ergast.com/api/f1/{requestContent}.{setupParameters['EXPORT']}?limit={setupParameters['LIMIT']}&offset={setupParameters['OFFSET']}"
    ErgastRequest = requests.get(requestURL, headers={}, data={})
    try:
        if ErgastRequest.status_code == 200:
            return ErgastRequest
        print(f'Request failed with status code {ErgastRequest.status_code}')
    except Exception as e:
        print(f'Error Caught while requesting the Ergast API: {e}')


# - Export a dictionnary as a JSON file - #
def exportJSON(dictContent: dict, filePath: str):
    """Export a dictionnary as a JSON file.
    Args:
        dictContent (dict): the dictionary to export.
        filePath (str): the path where to export the file.
    """    
    if not os.path.exists(os.path.dirname(filePath)):
        os.makedirs(os.path.dirname(filePath), exist_ok=True)
    with open(filePath, 'w') as fp:
        json.dump(dictContent, fp, indent=4)

---

#### Core - API Ergast

In [3]:
# 1° - Get the SEASONS
seasonsReqs = makeErgastRequest('seasons').json()
dictF1 = {seasonContent['season'] : dict() for seasonContent in seasonsReqs['MRData']['SeasonTable']['Seasons']}
displayDictSample(dictF1, 5)

Sample of the Dictionnary (5 observations):
******************************************* 

1977 {}
1986 {}
2002 {}
2022 {}
2023 {}


In [4]:
# 1.bis° - Get the CIRCUITS
circuitsReqs = makeErgastRequest('circuits').json()
circuitsF1 = dict()
for circuitContent in circuitsReqs['MRData']['CircuitTable']['Circuits']:
    circuitsF1[circuitContent['circuitId']] = dict({
        'CIRCUIT_NAME': circuitContent['circuitName'],
        'COUNTRY': circuitContent['Location']['country'],
        'CITY': circuitContent['Location']['locality'],
        'COORDINATES' : (circuitContent['Location']['lat'], circuitContent['Location']['long'])
    })
displayDictSample(circuitsF1, 5)

Sample of the Dictionnary (5 observations):
******************************************* 

americas {'CIRCUIT_NAME': 'Circuit of the Americas', 'COUNTRY': 'USA', 'CITY': 'Austin', 'COORDINATES': ('30.1328', '-97.6411')}
dallas {'CIRCUIT_NAME': 'Fair Park', 'COUNTRY': 'USA', 'CITY': 'Dallas', 'COORDINATES': ('32.7774', '-96.7587')}
george {'CIRCUIT_NAME': 'Prince George Circuit', 'COUNTRY': 'South Africa', 'CITY': 'Eastern Cape Province', 'COORDINATES': ('-33.0486', '27.8736')}
pescara {'CIRCUIT_NAME': 'Pescara Circuit', 'COUNTRY': 'Italy', 'CITY': 'Pescara', 'COORDINATES': ('42.475', '14.1508')}
zandvoort {'CIRCUIT_NAME': 'Circuit Park Zandvoort', 'COUNTRY': 'Netherlands', 'CITY': 'Zandvoort', 'COORDINATES': ('52.3888', '4.54092')}


In [283]:
# 1.bis2° - Get the DRIVERS
driversReqs = makeErgastRequest('drivers').json()
driversF1 = dict()
for driverContent in driversReqs['MRData']['DriverTable']['Drivers']:
    driversF1[driverContent['driverId']] = dict({
        'GIVEN_NAME': driverContent['givenName'],
        'FAMILY_NAME': driverContent['familyName'],
        'DATE_OF_BIRTH': driverContent['dateOfBirth'],
        'NATIONALITY': driverContent['nationality'],
        'URL': driverContent['url']
    })
displayDictSample(driversF1, 5)

Sample of the Dictionnary (5 observations):
******************************************* 

branson {'GIVEN_NAME': 'Don', 'FAMILY_NAME': 'Branson', 'DATE_OF_BIRTH': '1920-06-02', 'NATIONALITY': 'American', 'URL': 'http://en.wikipedia.org/wiki/Don_Branson'}
grim {'GIVEN_NAME': 'Bobby', 'FAMILY_NAME': 'Grim', 'DATE_OF_BIRTH': '1924-09-04', 'NATIONALITY': 'American', 'URL': 'http://en.wikipedia.org/wiki/Bobby_Grim'}
leclerc {'GIVEN_NAME': 'Charles', 'FAMILY_NAME': 'Leclerc', 'DATE_OF_BIRTH': '1997-10-16', 'NATIONALITY': 'Monegasque', 'URL': 'http://en.wikipedia.org/wiki/Charles_Leclerc'}
rathmann {'GIVEN_NAME': 'Jim', 'FAMILY_NAME': 'Rathmann', 'DATE_OF_BIRTH': '1928-07-16', 'NATIONALITY': 'American', 'URL': 'http://en.wikipedia.org/wiki/Jim_Rathmann'}
ratzenberger {'GIVEN_NAME': 'Roland', 'FAMILY_NAME': 'Ratzenberger', 'DATE_OF_BIRTH': '1960-07-04', 'NATIONALITY': 'Austrian', 'URL': 'http://en.wikipedia.org/wiki/Roland_Ratzenberger'}


In [286]:
# 1.bis3° - Get the CONSTRUCTORS
constructorsReqs = makeErgastRequest('constructors').json()
constructorsF1 = dict()
for constructorContent in constructorsReqs['MRData']['ConstructorTable']['Constructors']:
    constructorsF1[constructorContent['constructorId']] = dict({
        'NAME': constructorContent['name'],
        'NATIONALITY': constructorContent['nationality'],
        'URL': constructorContent['url']
    })
displayDictSample(constructorsF1, 5)

Sample of the Dictionnary (5 observations):
******************************************* 

bmw {'NAME': 'BMW', 'NATIONALITY': 'German', 'URL': 'http://en.wikipedia.org/wiki/BMW'}
brawn {'NAME': 'Brawn', 'NATIONALITY': 'British', 'URL': 'http://en.wikipedia.org/wiki/Brawn_GP'}
cooper-brm {'NAME': 'Cooper-BRM', 'NATIONALITY': 'British', 'URL': 'http://en.wikipedia.org/wiki/Cooper_Car_Company'}
ensign {'NAME': 'Ensign', 'NATIONALITY': 'British', 'URL': 'http://en.wikipedia.org/wiki/Ensign_%28racing_team%29'}
porsche {'NAME': 'Porsche', 'NATIONALITY': 'German', 'URL': 'http://en.wikipedia.org/wiki/Porsche_in_Formula_One'}


In [ ]:
# 2.1° - Get the DRIVERS per SEASON
for seasonKey in dictF1.keys():
    driverReqs = makeErgastRequest(f'{seasonKey}/drivers').json()
    dictF1[seasonKey]['DRIVERS'] = dict()
    for driverContent in driverReqs['MRData']['DriverTable']['Drivers']:
        dictDriver = dict()
        dictDriver = updateDictionnary(driverContent, dictDriver, ['permanentNumber', 'code', 'givenName', 'familyName', 'dateOfBirth', 'nationality'], ['PERMANENT_NUMBER', 'CODE', 'FIRST_NAME', 'LAST_NAME', 'DATE_OF_BIRTH', 'NATIONALITY'])
        dictF1[seasonKey]['DRIVERS'][driverContent['driverId']] = dictDriver
displayDictSample(dictF1[random.choice(list(dictF1.keys()))]['DRIVERS'], 5)

In [6]:
# 2.2° - Get the CONSTRUCTORS per SEASON
for seasonKey in dictF1.keys():
    constructorReqs = makeErgastRequest(f'{seasonKey}/constructors').json()
    dictF1[seasonKey]['CONSTRUCTORS'] = dict()
    for constructorContent in constructorReqs['MRData']['ConstructorTable']['Constructors']:
        dictConstructor = dict()
        dictConstructor = updateDictionnary(constructorContent, dictConstructor, ['name', 'nationality'], ['NAME', 'NATIONALITY'])
        dictF1[seasonKey]['CONSTRUCTORS'][constructorContent['constructorId']] = dictConstructor
displayDictSample(dictF1[random.choice(list(dictF1.keys()))]['CONSTRUCTORS'], 5)

Sample of the Dictionnary (5 observations):
******************************************* 

force_india {'NAME': 'Force India', 'NATIONALITY': 'Indian'}
lotus_racing {'NAME': 'Lotus', 'NATIONALITY': 'Malaysian'}
mclaren {'NAME': 'McLaren', 'NATIONALITY': 'British'}
mercedes {'NAME': 'Mercedes', 'NATIONALITY': 'German'}
virgin {'NAME': 'Virgin', 'NATIONALITY': 'British'}


In [7]:
# 2.3° - Get the RACES per SEASON
for seasonKey in dictF1.keys():
    raceReqs = makeErgastRequest(f'{seasonKey}').json()
    dictF1[seasonKey]['RACES'] = dict()
    for raceContent in raceReqs['MRData']['RaceTable']['Races']:
        dictRace = dict()
        dictRace = updateDictionnary(raceContent['Circuit'], dictRace, ['circuitId'], ['CIRCUIT_ID'])
        dictRace['RACE'] = dict()
        dictRace['RACE'] = updateDictionnary(raceContent, dictRace['RACE'], ['raceName', 'date', 'time'], ['NAME', 'DATE', 'TIME'])
        dictRace = updateDictionnary(raceContent, dictRace, ['FirstPractice', 'SecondPractice', 'ThirdPractice', 'Sprint'], ['PRACTICE_1', 'PRACTICE_2', 'PRACTICE_3', 'SPRINT'])
        dictF1[seasonKey]['RACES'][raceContent['round']] = dictRace
displayDictSample(dictF1[random.choice(list(dictF1.keys()))]['RACES'], 5)

Sample of the Dictionnary (5 observations):
******************************************* 

10 {'CIRCUIT_ID': 'silverstone', 'RACE': {'NAME': 'British Grand Prix', 'DATE': '2019-07-14', 'TIME': '13:10:00Z'}}
11 {'CIRCUIT_ID': 'hockenheimring', 'RACE': {'NAME': 'German Grand Prix', 'DATE': '2019-07-28', 'TIME': '13:10:00Z'}}
17 {'CIRCUIT_ID': 'suzuka', 'RACE': {'NAME': 'Japanese Grand Prix', 'DATE': '2019-10-13', 'TIME': '05:10:00Z'}}
2 {'CIRCUIT_ID': 'bahrain', 'RACE': {'NAME': 'Bahrain Grand Prix', 'DATE': '2019-03-31', 'TIME': '15:10:00Z'}}
7 {'CIRCUIT_ID': 'villeneuve', 'RACE': {'NAME': 'Canadian Grand Prix', 'DATE': '2019-06-09', 'TIME': '18:10:00Z'}}


In [8]:
# 2.4° - Get the RACES RESULTS per SEASON
for seasonKey in dictF1.keys():
    resultReqs = makeErgastRequest(f'{seasonKey}/results').json()
    dictF1[seasonKey]['RESULTS'] = dict()
    for resultContent in resultReqs['MRData']['RaceTable']['Races']:
        dictResult = dict()
        for driverContent in resultContent['Results']:
            dictResult[driverContent['position']] = dict()
            dictResult[driverContent['position']] = updateDictionnary(driverContent['Driver'], dictResult[driverContent['position']], ['driverId'], ['DRIVER_ID'])
            dictResult[driverContent['position']] = updateDictionnary(driverContent['Constructor'], dictResult[driverContent['position']], ['constructorId'], ['CONSTRUCTOR_ID'])
            dictResult[driverContent['position']] = updateDictionnary(driverContent, dictResult[driverContent['position']], ['grid', 'status', 'lap', 'Time', 'FastestLap', 'AverageSpeed'], ['GRID_POSITION', 'STATUS', 'N_LAPS', 'TIME', 'FASTEST_LAP', 'AVERAGE_SPEED'])
            dictF1[seasonKey]['RESULTS'][resultContent['round']] = dictResult
displayDictSample(dictF1[random.choice(list(dictF1.keys()))]['RESULTS'], 5)

Sample of the Dictionnary (5 observations):
******************************************* 

2 {'1': {'DRIVER_ID': 'ward', 'CONSTRUCTOR_ID': 'watson', 'GRID_POSITION': '6', 'STATUS': 'Finished', 'TIME': {'millis': '13249200', 'time': '3:40:49.20'}}, '2': {'DRIVER_ID': 'rathmann', 'CONSTRUCTOR_ID': 'watson', 'GRID_POSITION': '3', 'STATUS': 'Finished', 'TIME': {'millis': '13272480', 'time': '+0:23.28'}}, '3': {'DRIVER_ID': 'thomson', 'CONSTRUCTOR_ID': 'lesovsky', 'GRID_POSITION': '1', 'STATUS': 'Finished', 'TIME': {'millis': '13299840', 'time': '+0:50.64'}}, '4': {'DRIVER_ID': 'bettenhausen', 'CONSTRUCTOR_ID': 'epperly', 'GRID_POSITION': '15', 'STATUS': 'Finished', 'TIME': {'millis': '13356290', 'time': '+1:47.09'}}, '5': {'DRIVER_ID': 'goldsmith', 'CONSTRUCTOR_ID': 'epperly', 'GRID_POSITION': '16', 'STATUS': 'Finished', 'TIME': {'millis': '13375640', 'time': '+2:06.44'}}, '6': {'DRIVER_ID': 'boyd', 'CONSTRUCTOR_ID': 'epperly', 'GRID_POSITION': '11', 'STATUS': 'Finished', 'TIME': {'millis':

In [9]:
# 2.5° - Get the RACES RESULTS per LAP per SEASON
for seasonKey in dictF1.keys():
    dictF1[seasonKey]['LAPS'] = dict()
    n_races = len(makeErgastRequest(f'{seasonKey}').json()['MRData']['RaceTable']['Races'])
    for raceNum in range(1, n_races+1):       
        lapReqs = makeErgastRequest(f'{seasonKey}/{raceNum}/laps').json()

        if not lapReqs['MRData']['RaceTable']['Races']:
            dictF1[seasonKey]['LAPS'][raceNum-1] = dict()
            continue
        
        dictLaps = dict()
        for lapsContent in lapReqs['MRData']['RaceTable']['Races'][0]['Laps']:
            dictLaps[lapsContent['number']] = dict()
            for driverContent in lapsContent['Timings']:
                dictLaps[lapsContent['number']][driverContent['driverId']] = dict()
                dictLaps[lapsContent['number']][driverContent['driverId']] = updateDictionnary(driverContent, dictLaps[lapsContent['number']][driverContent['driverId']], ['position', 'time'], ['POSITION', 'TIME'])
        dictF1[seasonKey]['LAPS'][raceNum] = dictLaps
displayDictSample(dictF1[random.choice(list(dictF1.keys()))]['LAPS'], 5)

Sample of the Dictionnary (5 observations):
******************************************* 

4 {'1': {'barrichello': {'POSITION': '1', 'TIME': '1:33.940'}, 'frentzen': {'POSITION': '2', 'TIME': '1:34.369'}, 'coulthard': {'POSITION': '3', 'TIME': '1:34.818'}, 'hakkinen': {'POSITION': '4', 'TIME': '1:35.485'}, 'button': {'POSITION': '5', 'TIME': '1:35.935'}, 'villeneuve': {'POSITION': '6', 'TIME': '1:37.208'}, 'ralf_schumacher': {'POSITION': '7', 'TIME': '1:38.012'}, 'michael_schumacher': {'POSITION': '8', 'TIME': '1:38.881'}, 'verstappen': {'POSITION': '9', 'TIME': '1:39.647'}, 'trulli': {'POSITION': '10', 'TIME': '1:40.168'}, 'fisichella': {'POSITION': '11', 'TIME': '1:40.518'}, 'irvine': {'POSITION': '12', 'TIME': '1:40.810'}, 'alesi': {'POSITION': '13', 'TIME': '1:41.302'}, 'salo': {'POSITION': '14', 'TIME': '1:41.714'}, 'zonta': {'POSITION': '15', 'TIME': '1:42.110'}, 'diniz': {'POSITION': '16', 'TIME': '1:42.746'}, 'rosa': {'POSITION': '17', 'TIME': '1:43.208'}, 'wurz': {'POSITION': '

In [251]:
# 3° - Get the QUALIFYINGS RESULTS per SEASON
lsSeasons = dictF1.keys()
dictQualifying = dict()
for season in list(lsSeasons):
    dictQualifying[season] = dict()
    
    # -- Request the Qualifying Results -- #
    qualifyingReqs = getInnerDict(makeErgastRequest(f'{season}/qualifying').json(), ['MRData', 'RaceTable', 'Races'])
    if not qualifyingReqs:  # If there are no qualifying results, we skip the season
        continue
    
    # -- Loop over all Races -- #
    for raceContent in qualifyingReqs:
        dictQualifying[season][raceContent['round']] = dict({
            'RACE_NAME': raceContent['raceName'],
            'CIRCUIT': raceContent['Circuit']['circuitId'],
            'DATE': raceContent['date'],
            'RESULTS': []
        })
        
        # -- Loop over all Drivers Qualifying Results -- #
        for res in raceContent['QualifyingResults']:
            dictQualifying[season][raceContent['round']]['RESULTS'].append(dict({
                'POSITION': res['position'],
                'DRIVER': res['Driver']['driverId'],
                'CONSTRUCTOR': res['Constructor']['constructorId'],
                'Q1': getInnerDict(res,['Q1']),
                'Q2': getInnerDict(res, ['Q2']),
                'Q3': getInnerDict(res, ['Q3'])
            }))
            
exampleSeason = random.choice(list(filter(lambda season : dictQualifying[season].keys(), dictQualifying.keys())))
displayListSample(dictQualifying[exampleSeason][random.choice(list(dictQualifying[exampleSeason].keys()))]['RESULTS'], 5)

Sample of the List (5 observations):
************************************ 

{'POSITION': '1', 'DRIVER': 'coulthard', 'CONSTRUCTOR': 'mclaren', 'Q1': '1:25.852', 'Q2': '', 'Q3': ''}
{'POSITION': '2', 'DRIVER': 'michael_schumacher', 'CONSTRUCTOR': 'ferrari', 'Q1': '1:26.251', 'Q2': '', 'Q3': ''}
{'POSITION': '3', 'DRIVER': 'hakkinen', 'CONSTRUCTOR': 'mclaren', 'Q1': '1:26.632', 'Q2': '', 'Q3': ''}
{'POSITION': '4', 'DRIVER': 'irvine', 'CONSTRUCTOR': 'ferrari', 'Q1': '1:26.780', 'Q2': '', 'Q3': ''}
{'POSITION': '5', 'DRIVER': 'ralf_schumacher', 'CONSTRUCTOR': 'jordan', 'Q1': '1:26.827', 'Q2': '', 'Q3': ''}


In [250]:
# 4° - Get the SPRINT QUALIFYINGS RESULTS per SEASON
lsSeasons = dictF1.keys()
dictSprint = dict()
for season in list(lsSeasons):
    dictSprint[season] = dict()
    
    # -- Request the Qualifying Results -- #
    sprintReqs = getInnerDict(makeErgastRequest(f'{season}/sprint').json(), ['MRData', 'RaceTable', 'Races'])
    if not qualifyingReqs:  # If there are no qualifying results, we skip the season
        continue
    
    # -- Loop over all Races -- #
    for raceContent in sprintReqs:
        dictSprint[season][raceContent['round']] = dict({
            'RACE_NAME': getInnerDict(raceContent, ['raceName']),
            'CIRCUIT': getInnerDict(raceContent, ['Circuit', 'circuitId']),
            'DATE': getInnerDict(raceContent, ['date']),
            'RESULTS': []
        })
        
        # -- Loop over all Drivers Qualifying Results -- #
        for res in raceContent['SprintResults']:
            dictSprint[season][raceContent['round']]['RESULTS'].append(dict({
                'STATUS': getInnerDict(res, ['status']),
                'POSITION': getInnerDict(res, ['position']),
                'GRID': getInnerDict(res, ['grid']),
                'LAPS': getInnerDict(res, ['laps']),
                'TIME': getInnerDict(res, ['Time', 'time']),
                'FASTEST_LAP': {'LAP': getInnerDict(res, ['FastestLap', 'lap']), 'TIME': getInnerDict(res, ['FastestLap', 'Time', 'time'])},
                'POINT': getInnerDict(res, ['points']),
                'DRIVER': getInnerDict(res, ['Driver', 'driverId']),
                'CONSTRUCTOR': getInnerDict(res, ['Constructor', 'constructorId']),
            }))
            
exampleSeason = random.choice(list(filter(lambda season : dictSprint[season].keys(), dictSprint.keys())))
displayListSample(dictSprint[exampleSeason][random.choice(list(dictSprint[exampleSeason].keys()))]['RESULTS'], 5)

Sample of the List (5 observations):
************************************ 

{'STATUS': 'Finished', 'POSITION': '1', 'GRID': '1', 'LAPS': '18', 'TIME': '27:54.078', 'FASTEST_LAP': {'LAP': '9', 'TIME': '1:23.540'}, 'POINT': '3', 'DRIVER': 'bottas', 'CONSTRUCTOR': 'mercedes'}
{'STATUS': 'Finished', 'POSITION': '2', 'GRID': '3', 'LAPS': '18', 'TIME': '+2.325', 'FASTEST_LAP': {'LAP': '9', 'TIME': '1:23.502'}, 'POINT': '2', 'DRIVER': 'max_verstappen', 'CONSTRUCTOR': 'red_bull'}
{'STATUS': 'Finished', 'POSITION': '3', 'GRID': '5', 'LAPS': '18', 'TIME': '+14.534', 'FASTEST_LAP': {'LAP': '17', 'TIME': '1:24.291'}, 'POINT': '1', 'DRIVER': 'ricciardo', 'CONSTRUCTOR': 'mclaren'}
{'STATUS': 'Finished', 'POSITION': '4', 'GRID': '4', 'LAPS': '18', 'TIME': '+18.835', 'FASTEST_LAP': {'LAP': '9', 'TIME': '1:24.554'}, 'POINT': '0', 'DRIVER': 'norris', 'CONSTRUCTOR': 'mclaren'}
{'STATUS': 'Finished', 'POSITION': '5', 'GRID': '2', 'LAPS': '18', 'TIME': '+20.011', 'FASTEST_LAP': {'LAP': '9', 'TIME': '1:24.2

In [260]:
# 5° - Get the RACE SCHEDULE 
lsSeasons = dictF1.keys()
dictSchedule = dict()
for season in list(lsSeasons):
    dictSchedule[season] = dict()
    
    # -- Request the Qualifying Results -- #
    scheduleReqs = getInnerDict(makeErgastRequest(f'{season}').json(), ['MRData', 'RaceTable', 'Races'])
    if not scheduleReqs:  # If there are no schedule results, we skip the season
        continue
    
    # -- Loop over all Races -- #
    for raceContent in scheduleReqs:
        dictSchedule[season][raceContent['round']] = dict({
            'RACE_NAME': getInnerDict(raceContent, ['raceName']),
            'CIRCUIT': getInnerDict(raceContent, ['Circuit', 'circuitId']),
            'DATE': getInnerDict(raceContent, ['date']),
            'TIME': getInnerDict(raceContent, ['time']),
            'FirstPractice': {'DATE': getInnerDict(raceContent, ['FirstPractice', 'date']), 'TIME': getInnerDict(raceContent, ['FirstPractice', 'time'])},
            'SecondPractice': {'DATE': getInnerDict(raceContent, ['SecondPractice', 'date']), 'TIME': getInnerDict(raceContent, ['SecondPractice', 'time'])},
            'ThirdPractice': {'DATE': getInnerDict(raceContent, ['ThirdPractice', 'date']), 'TIME': getInnerDict(raceContent, ['ThirdPractice', 'time'])},
            'Qualifying': {'DATE': getInnerDict(raceContent, ['Qualifying', 'date']), 'TIME': getInnerDict(raceContent, ['Qualifying', 'time'])},
            'URL': getInnerDict(raceContent, ['url']),
        })
        
displayDictSample(dictSchedule[random.choice(list(dictSchedule.keys()))], 5)

Sample of the Dictionnary (5 observations):
******************************************* 

12 {'RACE_NAME': 'Turkish Grand Prix', 'CIRCUIT': 'istanbul', 'DATE': '2007-08-26', 'TIME': '12:00:00Z', 'FirstPractice': {'DATE': '', 'TIME': ''}, 'SecondPractice': {'DATE': '', 'TIME': ''}, 'ThirdPractice': {'DATE': '', 'TIME': ''}, 'Qualifying': {'DATE': '', 'TIME': ''}, 'URL': 'http://en.wikipedia.org/wiki/2007_Turkish_Grand_Prix'}
16 {'RACE_NAME': 'Chinese Grand Prix', 'CIRCUIT': 'shanghai', 'DATE': '2007-10-07', 'TIME': '06:00:00Z', 'FirstPractice': {'DATE': '', 'TIME': ''}, 'SecondPractice': {'DATE': '', 'TIME': ''}, 'ThirdPractice': {'DATE': '', 'TIME': ''}, 'Qualifying': {'DATE': '', 'TIME': ''}, 'URL': 'http://en.wikipedia.org/wiki/2007_Chinese_Grand_Prix'}
3 {'RACE_NAME': 'Bahrain Grand Prix', 'CIRCUIT': 'bahrain', 'DATE': '2007-04-15', 'TIME': '11:30:00Z', 'FirstPractice': {'DATE': '', 'TIME': ''}, 'SecondPractice': {'DATE': '', 'TIME': ''}, 'ThirdPractice': {'DATE': '', 'TIME': ''}, '

In [ ]:
# 6° - Get the DRIVERS STANDINGS per SEASON (OPTIONAL)
# ~2mn per season depending on the request time as the API require to call for each race of every season.
# - Uncomment the following code to get the drivers standings per season - #

# lsSeasons = dictSchedule.keys()
# dictStandingDriver = dict()
# for raceSeason in list(lsSeasons):
#     dictStandingDriver[raceSeason] = dict({})
#     for raceRound in list(getInnerDict(dictSchedule, [season]).keys()):
#         dictStandingDriver[raceSeason][raceRound] = dict({})
    
#         # -- Request the Qualifying Results -- #
#         driverStandingReqs = getInnerDict(makeErgastRequest(f'{raceSeason}/{raceRound}/driverStandings').json(), ['MRData', 'StandingsTable', 'StandingsLists'])
#         if not driverStandingReqs:  # If there are no qualifying results, we skip the season
#             continue
        
#         # -- Loop over all Drivers -- #
#         for res in getInnerDict(driverStandingReqs[0], ['DriverStandings']):
#             dictStandingDriver[raceSeason][raceRound][getInnerDict(res, ['position'])] = dict({
#                 'POSITION': getInnerDict(res, ['position']),
#                 'POINTS': getInnerDict(res, ['points']),
#                 'WINS': getInnerDict(res, ['wins']),
#                 'DRIVER': getInnerDict(res, ['Driver', 'driverId']),
#                 'CONSTRUCTOR': [getInnerDict(cstr, ['constructorId']) for cstr in getInnerDict(res, ['Constructors'])]
#             })

# exampleSeason = random.choice(list(filter(lambda season : len((list(itertools.chain(*[value for _, value in dictStandingDriver[season].items()])))) > 0, dictStandingDriver.keys())))
# displayDictSample(dictStandingDriver[exampleSeason][random.choice(list(dictStandingDriver[exampleSeason].keys()))], 5)

In [ ]:
# 7° - Get the CONSTRUCTORS STANDINGS per SEASON (OPTIONAL)
# ~10s per season depending on the request time as the API require to call for each race of every season.
# - Uncomment the following code to get the drivers standings per season - #

# lsSeasons = dictSchedule.keys()
# dictStandingConstructor = dict()
# for raceSeason in list(lsSeasons):
#     dictStandingConstructor[raceSeason] = dict({})
#     for raceRound in list(getInnerDict(dictSchedule, [season]).keys()):
#         dictStandingConstructor[raceSeason][raceRound] = dict({})
    
#         # -- Request the Qualifying Results -- #
#         constructorsStandingReqs = getInnerDict(makeErgastRequest(f'{raceSeason}/{raceRound}/constructorStandings').json(), ['MRData', 'StandingsTable', 'StandingsLists'])
#         if not constructorsStandingReqs:  # If there are no qualifying results, we skip the season
#             continue
        
#         # -- Loop over all Drivers -- #
#         for res in getInnerDict(constructorsStandingReqs[0], ['ConstructorStandings']):
#             dictStandingConstructor[raceSeason][raceRound][getInnerDict(res, ['position'])] = dict({
#                 'POSITION': getInnerDict(res, ['position']),
#                 'POINTS': getInnerDict(res, ['points']),
#                 'WINS': getInnerDict(res, ['wins']),
#                 'CONSTRUCTOR':  getInnerDict(res, ['Constructor', 'constructorId']),
#             })

# exampleSeason = random.choice(list(filter(lambda season : len((list(itertools.chain(*[value for _, value in dictStandingConstructor[season].items()])))) > 0, dictStandingConstructor.keys())))
# displayDictSample(dictStandingConstructor[exampleSeason][random.choice(list(dictStandingConstructor[exampleSeason].keys()))], 5)

In [ ]:
# 8° - Get the PITSTOPS per SEASON (OPTIONAL)
# ~10s per season depending on the request time as the API require to call for each race of every season.
# - Uncomment the following code to get the drivers standings per season - #

lsSeasons = dictSchedule.keys()
dictPitstops = dict()
for raceSeason in list(lsSeasons):
    print(raceSeason)
    dictPitstops[raceSeason] = dict({})
    for raceRound in list(getInnerDict(dictSchedule, [season]).keys()):
        dictPitstops[raceSeason][raceRound] = []
    
        # -- Request the Qualifying Results -- #
        pitstopsStandingReqs = getInnerDict(makeErgastRequest(f'{raceSeason}/{raceRound}/pitstops').json(), ['MRData', 'RaceTable', 'Races'])
        if not pitstopsStandingReqs:  # If there are no qualifying results, we skip the season
            continue
        
        # -- Loop over all Drivers -- #
        for res in getInnerDict(pitstopsStandingReqs[0], ['PitStops']):
            dictPitstops[raceSeason][raceRound].append(dict({
                'DRIVER': getInnerDict(res, ['driverId']),
                'LAP': getInnerDict(res, ['lap']),
                'TIME': getInnerDict(res, ['time']),
                'STOP_NUMBER': getInnerDict(res, ['stop']),
                'STOP_DURATION': getInnerDict(res, ['duration']),
            }))
            
exampleSeason = random.choice(list(filter(lambda season : len((list(itertools.chain(*[value for _, value in dictPitstops[season].items()])))) > 0, dictPitstops.keys())))
displayListSample(dictPitstops[exampleSeason][random.choice(list(dictPitstops[exampleSeason].keys()))], 5)

In [312]:
# 3° - Export the data in a JSON file
pathFile = os.path.join('../data', 'formula1-circuits.json')
exportJSON(circuitsF1, pathFile)

pathFile = os.path.join('../data', 'formula1-constructors.json')
exportJSON(constructorsF1, pathFile)

pathFile = os.path.join('../data', 'formula1-drivers.json')
exportJSON(driversF1, pathFile)
 
pathFile = os.path.join('../data', 'formula1-data.json')
exportJSON(dictF1, pathFile)

pathFile = os.path.join('../data', 'formula1-qualifying-results.json')
exportJSON(dictQualifying, pathFile)

pathFile = os.path.join('../data', 'formula1-sprints-qualifying-results.json')
exportJSON(dictSprint, pathFile)

pathFile = os.path.join('../data', 'formula1-schedule.json')
exportJSON(dictSchedule, pathFile)

# pathFile = os.path.join('../data', 'formula1-standings-drivers.json')
# exportJSON(dictStandingDriver, pathFile)

# pathFile = os.path.join('../data', 'formula1-standings-constructors.json')
# exportJSON(dictStandingConstructor, pathFile)

# pathFile = os.path.join('../data', 'formula1-pitstops.json')
# exportJSON(dictPitstops, pathFile)